# ARM and Networking

### Introduction

In [19]:
### Import required packages
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from apyori import apriori
import networkx as nx 

In [20]:
df = pd.read_csv("/Users/yangyilin/Desktop/anly-501-project-YilinYang2000-1/data/00-raw-data/tweetresult.csv")
df.head()

,Unnamed: 0,0
0,0,rt littlebirbmame aungaungnumber ตอบให้ตรงคำถา...
1,1,rt littlebirbmame yamyummy ประทับใจความดักดานม...
2,2,rt angel_packsss 🎉 promoções que estão rolando...
3,3,rt wayydaminn love privacy worth share
4,4,rt sapphnumberc hawaii island small population...


In [21]:
#USER PARAM
input_path			=	'/Users/yangyilin/Desktop/anly-501-project-YilinYang2000-1/data/00-raw-data/pytweetresult.csv'
compute_sentiment 	=	True		
sentiment    		=	[]			#average sentiment of each chunck of text 
ave_window_size		=	250			#size of scanning window for moving average
					

#OUTPUT FILE
output='transactions.txt'
if os.path.exists(output): os.remove(output)

#INITIALIZE
lemmatizer 	= 	WordNetLemmatizer()
ps 			=	PorterStemmer()
sia 		= 	SentimentIntensityAnalyzer()

#ADD MORE
stopwords	=	stopwords.words('english')
add=['mr','mrs','wa','dr','said','back','could','one','looked','like','know','around','dont']
for sp in add: stopwords.append(sp)

def read_and_clean(path,START=0,STOP=-1):
	global sentiment 

	#-----------------------
	#INSERT CODE TO READ IN AS ONE BIG STING
	file = open(path, 'rt')
	text = file.read().lower()
	file.close()
	#-----------------------

	#REMOVE HEADER, AND NEW LINES
	text=text.replace("'",'') #wasn't --> wasnt
	lines = text.splitlines(); text=''; 
	lines=lines[START:STOP]    # mystring.replace('\n', ' ')
	for line in lines: text=text+' '+line

	#-----------------------
	#INSERT CODE TO ONLY KEEP CHAR IN string.printable
	tmp = ''; printable = set(string.printable)
	for char in text:
		if(char in printable): tmp = tmp+char
	text = tmp
	#-----------------------

	#BREAK INTO CHUNKS (SENTANCES OR OTHERWISE)
	sentences=nltk.tokenize.sent_tokenize(text)  #SENTENCES

	#CLEAN AND LEMMATIZE
	keep='0123456789abcdefghijklmnopqrstuvwxy';

	new_sentences=[]; vocabulary=[]
	for sentence in sentences:
		new_sentence=''

		# REBUILD LEMITIZED SENTENCE
		for word in sentence.split():
			
			#ONLY KEEP CHAR IN "keep"
			tmp2=''
			for char in word: 
				if(char in keep): 
					tmp2=tmp2+char
				else:
					tmp2=tmp2+' '
			word=tmp2

			#-----------------------
			# INSERT CODE TO LEMMATIZE THE WORDS
			word = tmp2
			new_word = lemmatizer.lemmatize(word)
			#-----------------------

			#REMOVE WHITE SPACES
			new_word=new_word.replace(' ', '')

			#BUILD NEW SENTANCE BACK UP
			if( new_word not in stopwords):
				if(new_sentence==''):
					new_sentence=new_word
				else:
					new_sentence=new_sentence+','+new_word
				if(new_word not in vocabulary): vocabulary.append(new_word)

		#SAVE (LIST OF LISTS)		
		new_sentences.append(new_sentence.split(","))
		
		#SIA
		if(compute_sentiment):
			#-----------------------
			# INSERT CODE TO USE NLTK TO DO SENTIMENT ANALYSIS 
			text1 = new_sentence.replace(',', ' ')
			score = sia.polarity_scores(text1)
			sentiment.append([score['neg'], score['neu'], score['compound']])
			#-----------------------
			
		#SAVE SENTANCE TO OUTPUT FILE
		if(len(new_sentence.split(','))>2):
			f = open(output, "a")
			f.write(new_sentence+"\n")
			f.close()

	sentiment=np.array(sentiment)
	print("TOTAL AVERAGE SENTEMENT:",np.mean(sentiment,axis=0))
	print("VOCAB LENGTH",len(vocabulary))
	return new_sentences

transactions=read_and_clean(input_path,400,-400)
print(transactions[0:5])

TOTAL AVERAGE SENTEMENT: [0.06511549 0.80322254 0.12770817]
VOCAB LENGTH 4059
[['enter', 'rt', 'tweet', 'amp', 'tag', 'timpithenewway', 'amp', 'add', 'hashtag', 'timpipositive185', '172rt', 'honeymoonluve', 'gosta', 'da', 'viso'], ['set', 'desse', 'dia', 'com', 'muita', 'buceta', 'e', 'cuinho', 'l', 'privacy', 'view'], ['set', 'day', 'witpositive95', '173rt', 'littlebirbmame', 'yamyummy', 'unicef', 'un', 'privacy', 'privacy', 'neutral1000', '174rt', 'bblivery', 'privacy', 'neutral1000', '175rt', 'mahomet9', 'cant', 'even', 'stress', 'enough', 'privacy', 'crypto', 'active', 'process', 'constantly', 'work', 'maintain'], ['neutral784', '176rt', 'annabrennet', 'quartou', 'amores', 'privacy', 'atualiado', 'com', 'novo', 'pack', 'scat', '10', 'min', 'novos', 'assinantes', 'ganham', '15', 'min', 'de', 'chamada', '3', 'pack', 'rneutral1000', '177rt', 'johnmagrinho', 'kero', 'socar', 'um', 'cu', 'faer', 'de', 'buceta', 'bater', 'leite', 'espumar', 'quem', 'se', 'habilita'], ['assista', 'meus', 